In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun Mar 25 16:07:48 2018

@author: Griff
"""
from imdb import IMDb
import tmdbsimple as tmdb
import pandas as pd
import progressbar as prog
import os.path
import numpy as np
import IPython
from utilities_project import (get_raw_movies,cross_with_IMDb, split_string_columns,
                               encode_columns, split_lists_to_columns, MySQL_login, create_table)

In [2]:
#################
    ##SETUP
#################
print('Pandas version ' + pd.__version__)
print('tmdbsimple version ' + tmdb.__version__)

movie_count_req = 10000

tmdb.API_KEY = '43c26e89f50364998e6d406599d25aa5'
db = IMDb()
mysql, engine = MySQL_login()
cur = mysql.cursor()

Pandas version 0.22.0
tmdbsimple version 2.1.0


In [3]:
#################
    ##TMDb
#################    
TMDb_attributes = ['revenue','vote_average','vote_count'
                           ,'title','original_language','release_date'
                           ,'production_companies','production_countries','genres']

raw_movies = get_raw_movies(movie_count_req, TMDb_attributes)

if(raw_movies is not None):
    pd.DataFrame(data=raw_movies, columns = TMDb_attributes).to_csv('Movies.csv',index=False,header=True)

C:\Users\Griff\Documents\!_SKOOL\Database\Project\BrownGoresenLane\Movies.csv


In [4]:
#################
    ##IMDb
#################

def imdb_done():
    with open("movies.csv", encoding="utf8") as file:
        for line in file:
            if("director" in line):
                file.close()
                return True
        file.close()
        return False
    
if(not imdb_done()):
    IMDb_attributes = ["director","producer","cast","runtimes","writer"]
    raw_movies = pd.read_csv('Movies.csv', encoding="utf-8")
    raw_movies = cross_with_IMDb(raw_movies, IMDb_attributes)
    pd.DataFrame(data=raw_movies, columns = TMDb_attributes+IMDb_attributes).to_csv('Movies.csv',index=False,header=True)
else:
#     raw_movies = pd.read_sql('SELECT * FROM table_name', con=db_connection)
    raw_movies = pd.read_csv("Movies.csv", encoding="utf-8")

In [5]:
#################
  #PRE-CLEANING
#################
    
#Drop rows with columns we shouldn't impute
raw_movies = raw_movies.dropna(subset=["cast", "director", "vote_average", "vote_count","production_companies","release_date"])
raw_movies =raw_movies.reset_index()
print("dropped:",(movie_count_req-len(raw_movies)))

#Turn strings of multiple values into lists
#ignore_list  
test = split_string_columns(raw_movies, ignore_list=["title"])
print("done")

dropped 1538
done


In [6]:
test = split_lists_to_columns(engine, test, columns=["cast","producer","director","production_companies",
                                             "writer","production_countries","genres","original_language"],
                              embed = ["producer","director","cast","writer","production_companies"])

test = test[["title", "vote_average", "cast1", "cast2",
             "cast3", "producer1", "producer2", "producer3",
             "original_language1", "vote_count", "production_companies1",
             "production_companies2", "production_companies3", "director1",
             "director2", "director3", "revenue", "genres1","genres2","genres3", "release_date",
             "production_countries1","production_countries2","writer1", "runtimes"]]
print("done")

done


In [ ]:
create_table(test, "movie_data", engine)
print("done")

In [ ]:
#From server
stuff = pd.read_sql('SELECT * FROM movie_data', con=mysql).set_index("title")
print(stuff)